In [5]:
import numpy as np
import itertools
import time
import random
from collections import Counter, defaultdict

In [3]:
attributes = {
    'color': ['red', 'green', 'blue'],
    'fill': ['void', 'dashed', 'solid'],
    'shape': ['square', 'circle', 'triangle'],
    'config': ['XXO', 'XOX', 'OXX']
}

In [4]:
def generate_cards(attributes, attr_order):
    
    cards = []
    idx_to_card = {}
    card_to_idx = {}
    attrs_to_idx = defaultdict(lambda: defaultdict(list))

    i = 0
    attr_vals = [attributes[attr] for attr in attr_order]
    for combo in itertools.product(*attr_vals):
        card = tuple(combo)
        cards.append(card)
        card_to_idx[card] = i
        idx_to_card[i] = card
        i += 1
        for attr_val, attr_typ in zip(combo, attr_order):
            attrs_to_idx[attr_typ][attr_val].append(i)
    
    assert len(cards) == len(set(cards))
    print(f'Generated {len(cards)} unqiue cards')                
    return cards, idx_to_card, card_to_idx, attrs_to_idx


cards, idx_to_card, card_to_idx, attrs_to_idx = generate_cards(attributes, attr_order=('color', 'fill', 'shape', 'config'))
cards[:5]

Generated 81 unqiue cards


[('red', 'void', 'square', 'XXO'),
 ('red', 'void', 'square', 'XOX'),
 ('red', 'void', 'square', 'OXX'),
 ('red', 'void', 'circle', 'XXO'),
 ('red', 'void', 'circle', 'XOX')]

In [58]:
def generate_card_pairs(cards, card_to_idx):
    '''
    find all combos of cards, filter down to the ones that share concepts.
    '''
    cardpairs = []

    for card1, card2 in itertools.product(cards, cards):
        if card1 != card2:
            matching_concepts = tuple(s1 if s1==s2 else '-' for s1,s2 in zip(card1,card2))
            if set(matching_concepts) != {'-'}:
                cardpairs.append(((card_to_idx[card1], card_to_idx[card2]), matching_concepts))
    print(f'Generated {len(cardpairs)} unqiue cardpairs')  
    return cardpairs

cardpairs = generate_card_pairs(cards, card_to_idx)
cardpairs[:5]

Generated 5184 unqiue cardpairs


[((0, 1), ('red', 'void', 'square', '-')),
 ((0, 2), ('red', 'void', 'square', '-')),
 ((0, 3), ('red', 'void', '-', 'XXO')),
 ((0, 4), ('red', 'void', '-', '-')),
 ((0, 5), ('red', 'void', '-', '-'))]

In [50]:
def match_concept_to_card(concept, card):
    '''
    Given a concept, determine if card matches. Identify nonConcepts and violatedConcepts
    
    Arguments:
        concept: ('red', 'void', '-', '-')
        card: ex1. ('red', 'void', 'triangle', 'XOX')
              ex2. ('green', 'void', 'square', 'OXX')
    Returns:
        match: bool. ex1. True,
                     ex2. False
        nonConcept:  ex1. ('-', '-', 'triangle', 'XOX')
                     ex2. ('-', '-', 'square', 'OXX')
        violatedConcept:    ex1. ('-', '-', '-', '-')
                     ex2. ('red', '-', '-', '-')
    '''
    nonConcept = []
    violatedConcept = []
    match = True
    for ct, cd in zip(concept, card):
        if ct == '-':
            nonConcept.append(cd)
            violatedConcept.append('-')
        else:
            if cd != ct:
                match = False
                violatedConcept.append(ct)
            else:
                violatedConcept.append('-')
            nonConcept.append('-')

    return match, tuple(nonConcept), tuple(violatedConcept)
    
print(match_concept_to_card(concept=('red', 'void', '-', '-'), card=('red', 'void', 'triangle', 'XOX')))
print(match_concept_to_card(concept=('-', '-', 'triangle', 'XOX'), card=('red', 'void', 'triangle', 'XOX')))
print(match_concept_to_card(concept=('red', 'void', '-', '-'), card=('green', 'void', 'square', 'OXX')))
print(match_concept_to_card(concept=('red', 'void', '-', '-'), card=('green', 'solid', 'square', 'OXX')))

(True, ('-', '-', 'triangle', 'XOX'), ('-', '-', '-', '-'))
(True, ('red', 'void', '-', '-'), ('-', '-', '-', '-'))
(False, ('-', '-', 'square', 'OXX'), ('red', '-', '-', '-'))
(False, ('-', '-', 'square', 'OXX'), ('red', 'void', '-', '-'))


In [56]:
def find_notConcepts(concept, attr_order, attributes):
    '''
    Given a concept, find all its opposites.
    
    Arguments:
        concept: tuple. ex.('red', 'void', '-', '-')
        attr_order: tuple. ex.('colors', 'fills', 'shapes', 'configs')
    Returns:
        nonConcepts: list of tuples. ex.[
                 ('red', 'dashed', '-', '-')
                 ('red', 'solid', '-', '-')
                 ('green', 'void', '-', '-')
                 ('green', 'dashed', '-', '-')
                 ('green', 'solid', '-', '-')
                 ('blue', 'void', '-', '-')
                 ('blue', 'dashed', '-', '-')
                 ('blue', 'solid', '-', '-')                 
        ]
    '''
    notConcepts = set()
    
    # ['colors', 'fills']
    keep_attributes = list(attr for con, attr in zip(concept, attr_order) if con != '-')
    # [['red', 'green', 'blue'], ['void', 'dashed', 'solid'], ['-'], ['-']]
    keep_vals = [attributes[attr] if attr in keep_attributes else ['-'] for attr in attr_order]

    for combo in itertools.product(*keep_vals):
        if combo != concept:
            notConcepts.add(combo)
    
    return notConcepts

notConcepts = find_notConcepts(concept=('red', 'void', '-', '-'), attr_order=('color', 'fill', 'shape', 'config'), attributes=attributes)
notConcepts

{('blue', 'dashed', '-', '-'),
 ('blue', 'solid', '-', '-'),
 ('blue', 'void', '-', '-'),
 ('green', 'dashed', '-', '-'),
 ('green', 'solid', '-', '-'),
 ('green', 'void', '-', '-'),
 ('red', 'dashed', '-', '-'),
 ('red', 'solid', '-', '-')}

In [57]:
def make_negative_cards(notConcepts, nonConcept):
    negative_cards = []
    for notC, nonC in itertools.product(notConcepts, [nonConcept]):
        neg_card = tuple((x+y).strip('-') for x,y in zip(notC, nonC))
        negative_cards.append(neg_card)
    return negative_cards

make_negative_cards(notConcepts, nonConcept=('-', '-', 'triangle', 'XOX'))

[('green', 'solid', 'triangle', 'XOX'),
 ('blue', 'dashed', 'triangle', 'XOX'),
 ('blue', 'void', 'triangle', 'XOX'),
 ('green', 'dashed', 'triangle', 'XOX'),
 ('blue', 'solid', 'triangle', 'XOX'),
 ('red', 'solid', 'triangle', 'XOX'),
 ('red', 'dashed', 'triangle', 'XOX'),
 ('green', 'void', 'triangle', 'XOX')]

In [ ]:
cards, card_to_idx, attrs_to_idx = generate_cards(attributes, attr_order)
cardpairs = generate_card_pairs(cards, card_to_idx)

# 

In [ ]:
def match_concept_to_card(concept, cards):
    '''
    Given a concept, return subset of cards that match.
    
    Arguments:
        concept: ('red', 'void', '-', '-')
        cards: list of all cards.
    Returns:
        matching_cards: list of cards.
    '''
    matching_cards = []
    for card in cards:
        # True, ('-', '-', 'triangle', 'XOX'), ('-', '-', '-', '-')
        match_bool, nonConcept, violatedConcept = match_concept_to_card(concept, card)
        if match_bool:
            matching_cards.append(card)
    return matching_cards



In [ ]:
def gen_keys(cardpair):
    card_idx_pair, concept = cardpair
    pos_keys = 
    
    return pos_keys, neg_keys

In [ ]:

def gen_card_data(attributes, attr_order, num_val=100, num_test=100):
    
    cards, idx_to_card, card_to_idx, attrs_to_idx = generate_cards(attributes, attr_order)
    cardpairs = generate_card_pairs(cards, card_to_idx)
    
    random.shuffle(cardpairs)
    num_train = len(cardpairs) - num_val - num_test
    train_cardpairs = cardpairs[:num_train]
    valid_cardpairs = cardpairs[num_train:num_train+num_val]
    test_cardpairs = cardpairs[-num_test:]
    
    for cardpair in test_cardpairs:
    